1. http://www.manythings.org/anki 
 - 두 언어 간 병렬 코퍼스 다운 가능
 - en-fra : 10만개 이상 데이터 존재 -> 3만개만 사용
 - en-kor : 3천개 존재 -> 시도해봤는데 결과가 그닥...

2. 과제
 - 오늘 실습한 코드를 참고해서 peeky 모델 구현
 - 사용하고 싶은 데이터는 어떤 것을 사용해도 무관
 - 17기_OOO_seq2seq.ipynb 형태로 제출해주세요~

In [1]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
from google.colab import files 
files.upload() 

Output hidden; open in https://colab.research.google.com to view.

In [3]:
lines = pd.read_csv('fraa.txt', names=['src', 'tar', 'lic'], sep='\t',encoding='utf-8')
del lines['lic']
len(lines)

30000

In [4]:
lines.head()

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,Bouge !
3,Hi.,Salut !
4,Hi.,Salut.


## 1. Preprocessing

In [5]:
lines['src'] = lines['src'].apply(lambda x : re.sub(r"([?.!,¿])", r" \1", x))
lines['src'] = lines['src'].apply(lambda x : x.lower())
lines['tar'] = lines['tar'].apply(lambda x : re.sub(r"([?.!,¿])", r" \1", x))

lines.head()

,src,tar
0,go .,Va !
1,go .,Marche .
2,go .,Bouge !
3,hi .,Salut !
4,hi .,Salut .


In [6]:
src_input = []
for sample in lines['src'].to_numpy():
  src_input.append(sample.split())

tar_input = []
for sample in lines['tar'].to_numpy():
  tar_input.append(("<sos> " + sample).split())

tar_target = []
for sample in lines['tar'].to_numpy():
  tar_target.append((sample + " <eos>").split())

In [7]:
print(src_input[:5])
print(tar_input[:5])
print(tar_target[:5])

[['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.'], ['hi', '.']]
[['<sos>', 'Va', '!'], ['<sos>', 'Marche', '.'], ['<sos>', 'Bouge', '!'], ['<sos>', 'Salut', '!'], ['<sos>', 'Salut', '.']]
[['Va', '!', '<eos>'], ['Marche', '.', '<eos>'], ['Bouge', '!', '<eos>'], ['Salut', '!', '<eos>'], ['Salut', '.', '<eos>']]


In [8]:
tokenizer_src = Tokenizer()
tokenizer_src.fit_on_texts(src_input)
encoder_input = tokenizer_src.texts_to_sequences(src_input)

tokenizer_tar = Tokenizer()
tokenizer_tar.fit_on_texts(tar_input)
tokenizer_tar.fit_on_texts(tar_target)
decoder_input = tokenizer_tar.texts_to_sequences(tar_input)
decoder_target = tokenizer_tar.texts_to_sequences(tar_target)

In [9]:
src_vocab_size = len(tokenizer_src.word_index) + 1
tar_vocab_size = len(tokenizer_tar.word_index) + 1
print("src voca size : {:d}, tar voca size : {:d}".format(src_vocab_size, tar_vocab_size))

src voca size : 4470, tar voca size : 9344


In [10]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [11]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [12]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(30000, 7)
(30000, 14)
(30000, 14)


In [13]:
print(encoder_input[10])
print(decoder_input[10])
print(decoder_target[10])

[88  8 16 95  1  0  0]
[   2 4085    1    0    0    0    0    0    0    0    0    0    0    0]
[4085    1    3    0    0    0    0    0    0    0    0    0    0    0]


In [14]:
n_of_val = 300
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

## 2. Modeling

In [15]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

latent_dim = 50

In [16]:
# encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) 
encoder_lstm = LSTM(latent_dim, return_state=True) 
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb) 
encoder_states = [state_h, state_c] 

In [17]:
# decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) 
dec_emb = dec_emb_layer(decoder_inputs) 

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = encoder_states)

decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 32, epochs = 50)

Epoch 1/50
929/929 [==============================] - 47s 14ms/step - loss: 3.1177 - acc: 0.6511 - val_loss: 1.5680 - val_acc: 0.7610
Epoch 2/50
929/929 [==============================] - 12s 13ms/step - loss: 1.5543 - acc: 0.7658 - val_loss: 1.4120 - val_acc: 0.7883
Epoch 3/50
929/929 [==============================] - 12s 13ms/step - loss: 1.4052 - acc: 0.7887 - val_loss: 1.3198 - val_acc: 0.8043
Epoch 4/50
929/929 [==============================] - 12s 13ms/step - loss: 1.3139 - acc: 0.8037 - val_loss: 1.2675 - val_acc: 0.8088
Epoch 5/50
929/929 [==============================] - 12s 13ms/step - loss: 1.2640 - acc: 0.8123 - val_loss: 1.2248 - val_acc: 0.8133
Epoch 6/50
929/929 [==============================] - 12s 13ms/step - loss: 1.2184 - acc: 0.8176 - val_loss: 1.1977 - val_acc: 0.8183
Epoch 7/50
929/929 [==============================] - 12s 13ms/step - loss: 1.1923 - acc: 0.8213 - val_loss: 1.1868 - val_acc: 0.8219
Epoch 8/50
929/929 [==============================] - 12s 13ms

## 3. Model test

In [19]:
# encoder for test
encoder_model = Model(encoder_inputs, encoder_states)

# decoder for test
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [20]:
src_to_index = tokenizer_src.word_index
index_to_src = tokenizer_src.index_word 

tar_to_index = tokenizer_tar.word_index
index_to_tar = tokenizer_tar.index_word 

In [21]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    tar_seq = np.zeros((1,1))
    tar_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([tar_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, 0, :])
        sampled_char = index_to_tar[sampled_token_index]

        decoded_sentence += ' '+sampled_char

        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        tar_seq = np.zeros((1,1))
        tar_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

In [22]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

In [23]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input[seq_index]))
  print("번역문 :",seq2tar(decoder_input[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  who broke the cup ? 
번역문 : qui a cassé la tasse ? 
예측문 :  qui a la pas ? 


원문 :  that is mine . 
번역문 : c'est à moi . 
예측문 :  c'est le je . 


원문 :  are they worth it ? 
번역문 : valent-elles le coup ? 
예측문 :  le train de faire ? 


원문 :  i must go out . 
번역문 : je dois sortir . 
예측문 :  il me faut y aller . 


원문 :  the joke's on you . 
번역문 : c'est de vous qu'on se moque . 
예측문 :  on se la dit que ! 


